In [101]:
import folium
from io import BytesIO
import requests
import subprocess
#import warnings
#warnings.filterwarnings('ignore')

In [102]:
import hvplot.pandas
import pandas as pd

In [103]:
import requests

usgs_09058000 = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format"
                 "=rdb&site_no=09058000&legacy=&referred_module=sw&peri"
                 "od=&begin_date=1961-10-01&end_date=2022-09-30")
# Replace with the URL you want to request
# Send an HTTP GET request to the URL
mean_q_cfs = requests.get(usgs_09058000)

# Check if the request was successful (status code 200)
#if mean_q_cfs.status_code == 200:
    # Print the content of the response (HTML, JSON, etc.)
 #   print(mean_q_cfs.text)
#else:
#    print(f"Request failed with status code {mean_q_cfs.status_code}")

#mean_q_cfs

In [104]:
colorado_q_df = pd.read_csv(
    BytesIO(mean_q_cfs.content),
    comment='#',
    delimiter='\t',
    skiprows=[26,27,28,29],
    names=['agency_cd', 'site_no', 'datetime', 'discharge (cfs)', 'code'],
    index_col='datetime',
    parse_dates=True,
)
#colorado_q_df

In [105]:
colorado_annual_max_q_df = colorado_q_df[['discharge (cfs)']].resample('AS').max()
#colorado_annual_max_q_df

### Annual Maxuimum Peak Discharges Have Fluctuated Dramatically on the Colorado River near Kremmling, CO Over the Past 60 years

In [106]:
colorado_plot = colorado_annual_max_q_df.hvplot()
colorado_plot.opts(title='Annual Maximum Discharge Colorado River, Near Kremmling, Co')
colorado_plot

:Curve   [datetime]   (discharge (cfs))

In [107]:
colorado_annual_max_q_post_df = colorado_annual_max_q_df["1953":]
colorado_annual_max_q_post_df[
    "exceedance_prob"
] = colorado_annual_max_q_post_df.rank(ascending=False) / len(
    colorado_annual_max_q_post_df
)
colorado_annual_max_q_post_df["return_period"] = (
    1 / colorado_annual_max_q_post_df.exceedance_prob
)
#colorado_annual_max_q_post_df

### Kremmling Colorado has not seen a Large Flood since the 1960's

In [108]:
colorado_flood_return = colorado_annual_max_q_post_df.return_period.hvplot()
colorado_flood_return.opts(title='Flood Return Interval Colorado River, Near Kremmling, Co',ylabel='Return Interval (years)')
colorado_flood_return

:Curve   [datetime]   (return_period)

In [109]:
%%capture
%%bash
jupyter nbconvert colorado_Kremmling.ipynb --to html --no-input